In [1]:
import numpy as np
import os
import torch
from torch import nn, optim

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Using device", device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

Using device cuda:1
GeForce GTX 1660 SUPER
P106-100


In [2]:
from torch.utils.data import Dataset, DataLoader
from data.MinicityDataset import TrafficDataset

dbpath = '/home/rong/disk/database/minicity.db'
bs = 20
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [3]:
from models.minicity_nogoal_bigger import convVAE

X_dim = 4
traj_size = 25 * 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
print(model)

convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap_pool2): AdaptiveAvg

In [4]:
def loss_fn(recon_x, x, w, mean, log_var):
    MSE = torch.mean((w.expand_as(x) * (recon_x-x)**2))
    KLD = - 0.0005 * torch.mean(torch.sum(1 + log_var - mean.pow(2) - log_var.exp(), 1))
    return MSE + KLD, MSE

optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [5]:
def train(epoch, writer):
    model.train()
    train_loss = 0
    mse_loss = 0
    w = torch.tensor([8, 10, 2, 3], dtype=torch.float).to(device)
    adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
    
    for batch_idx, batch in enumerate(train_loader):
        startgoal = batch["start_goal"].to(device)
        startgoal[:, 4] = 0
        startgoal[:, 5] = 0
        occ = batch["observation"]
        occ = occ[:, :, 200:600, 200:600]        
        occ = adap_pool(occ)
        occ = occ.to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)
        traj = batch["traj"].to(device)

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, startgoal, traj,  occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        mse_loss += mse.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))
        
            writer.add_scalar('BatchLoss/loss', loss.item(), batch_idx)
            writer.add_scalar('BatchLoss/mse_loss', mse.item(), batch_idx)

    epoch_loss = train_loss * len(data) / len(train_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

def test(epoch):
    model.eval()
    test_loss = 0
    mse_loss = 0
    w = torch.tensor([8, 10, 2, 3], dtype=torch.float).to(device)
    for batch_idx, batch in enumerate(test_loader):
        startgoal = batch["start_goal"].to(device)
        occ = batch["observation"].to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)
        
        recon_batch, mu, logvar = model(sample, startend, occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        test_loss += loss.item()
        mse_loss += mse.item()

    epoch_loss = test_loss * len(data) / len(test_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(test_loader.dataset)
    print('====> Epoch: {} Average test loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

In [6]:
epoch = 0
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter('runs/minicity_attention_nogoal_2')

In [7]:
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model, device_ids=[1, 0])

model.to(device)

Let's use 2 GPUs!


/home/rong/.local/lib/python3.6/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


DataParallel(
  (module): convVAE(
    (condnn): CondNN(
      (cnn): Conv3d(
        (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
        (conv_layer1): Sequential(
          (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (3): LeakyReLU(negative_slope=0.01)
          (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (conv_layer2): Sequential(
          (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (3): LeakyReLU(negative_slope=0.01)
          (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (conv_layer5): Conv3d(32, 64, kernel_siz

In [8]:
for epoch in range(epoch, epoch + 200):
    log_interval = 20
    epoch, epoch_loss, epoch_mse = train(epoch, writer)
#     vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_loss]).unsqueeze(0).cpu(),win=loss_window,update='append',name='loss')
#         vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_mse]).unsqueeze(0).cpu(),win=loss_window,update='append',name='mse_loss')
    writer.add_scalar('Loss/loss', epoch_loss, epoch)
    writer.add_scalar('Loss/mse_loss', epoch_mse, epoch)

Train Epoch: 0 [0/210000 (0%)]	Loss: 0.134827
Train Epoch: 0 [400/210000 (0%)]	Loss: 0.096139
Train Epoch: 0 [800/210000 (0%)]	Loss: 0.031028
Train Epoch: 0 [1200/210000 (1%)]	Loss: 0.031510
Train Epoch: 0 [1600/210000 (1%)]	Loss: 0.056573
Train Epoch: 0 [2000/210000 (1%)]	Loss: 0.049827
Train Epoch: 0 [2400/210000 (1%)]	Loss: 0.032613
Train Epoch: 0 [2800/210000 (1%)]	Loss: 0.034210
Train Epoch: 0 [3200/210000 (2%)]	Loss: 0.028985
Train Epoch: 0 [3600/210000 (2%)]	Loss: 0.055694
Train Epoch: 0 [4000/210000 (2%)]	Loss: 0.031094
Train Epoch: 0 [4400/210000 (2%)]	Loss: 0.066184
Train Epoch: 0 [4800/210000 (2%)]	Loss: 0.023478
Train Epoch: 0 [5200/210000 (2%)]	Loss: 0.045533
Train Epoch: 0 [5600/210000 (3%)]	Loss: 0.037847
Train Epoch: 0 [6000/210000 (3%)]	Loss: 0.038264
Train Epoch: 0 [6400/210000 (3%)]	Loss: 0.049728
Train Epoch: 0 [6800/210000 (3%)]	Loss: 0.021330
Train Epoch: 0 [7200/210000 (3%)]	Loss: 0.022804
Train Epoch: 0 [7600/210000 (4%)]	Loss: 0.022089
Train Epoch: 0 [8000/2100

KeyboardInterrupt: 

In [9]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'checkpoints/minicity_attention_no_goal_2.pt')

In [10]:
torch.cuda.empty_cache()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("Using device", device)

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model, device_ids=[1, 0])

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

checkpoint = torch.load('checkpoints/minicity_attention_no_goal_2.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

Using device cuda:1
Let's use 2 GPUs!
DataParallel(
  (module): convVAE(
    (condnn): CondNN(
      (cnn): Conv3d(
        (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
        (conv_layer1): Sequential(
          (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (3): LeakyReLU(negative_slope=0.01)
          (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (conv_layer2): Sequential(
          (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
          (1): LeakyReLU(negative_slope=0.01)
          (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (3): LeakyReLU(negative_slope=0.01)
          (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (c

In [11]:
model = model.module

In [30]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
viz_idx = 8753 #弯道
# viz_idx = 143300 #弯道
# viz_idx = 5107 #直行
# viz_idx = 2984 #转盘
# viz_idx = 101538 #转盘
# viz_idx = 153309 #阻塞
# viz_idx = 109779 #变道

print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
startgoal[4] = 0
startgoal[5] = 0
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
occ = occ.to(device)
traj = torch.from_numpy(batch["traj"]).to(device)
print(traj.shape)
data = torch.from_numpy(batch["data"]).to(device)

with torch.no_grad():
    model.eval()
    y_viz = torch.randn(1,4).to(device)
    for i in range(0, 20):
        num_viz = 10
        y_viz_p, alpha = model.inference(startgoal.expand(num_viz, X_dim * 2).to(device), traj.expand(num_viz, 25, 4),
                                occ.expand(num_viz, 1, -1, -1, -1).to(device), num_viz)
        torch.cuda.empty_cache()
        y_viz = torch.cat((y_viz_p, y_viz), dim = 0)

y_viz=y_viz.cpu().detach().numpy()*50
occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
data=data.cpu().detach().numpy() * 50
alpha=alpha.cpu().detach().numpy()
torch.cuda.empty_cache()
# from utils.NarrowPassage import plotCondition, plotSample, plotSpeed, plotSampleAttention

%matplotlib
# from utils.HighWay import plotData, plotOrientSpeed, plotAlpha
from utils.Minicity import plotData, plotOrientSpeed, plotAlpha

y_viz=y_viz[:-1]
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
# plotAlpha(alpha)

8753
torch.Size([25, 4])
Using matplotlib backend: Qt5Agg
[ 0.      0.     -1.4532  9.86    0.      0.     -0.3976 11.98  ]
100.0 100.0


In [29]:
def plotAlpha(alpha):
    fig = plt.figure()
    alpha = alpha[0,:]
    alpha=alpha.reshape(4,10,10)
    
    myobj = plt.imshow(alpha[0, :, :])

    for frame in alpha:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.3)
plotAlpha(alpha)

In [20]:
import matplotlib.pyplot as plt

startx = 100
starty = 100

def position2imagep(data, startgoal):
    datax = (data[0])*(10/3) + 100
    datay = 100-(data[1])*(10/3) 
    return datax, datay

def plotCondition(occ, startgoal):
    fig = plt.figure()
    occ=occ.squeeze(0)
    occ=occ.squeeze(0)
    myobj = plt.imshow(occ[0, :, :])
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    plt.scatter(startx, starty)
    plt.scatter(goalx, goaly)
    for frame in occ:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.1)

def plotData(occ, startgoal,data):
    fig = plt.figure()
    occ=occ.squeeze(0)
    occ=occ.squeeze(0)
    myobj = plt.imshow(occ[0, :, :])
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    print(startgoal)
    print(goalx, goaly)
    plt.scatter(startx, starty)
    plt.scatter(goalx, goaly)
    if len(data.shape) > 1:
        for row in data:
            datax, datay = position2imagep(row, startgoal)
            plt.scatter(datax, datay, marker='*', c='#d62728')
    else:
        datax, datay = position2imagep(data, startgoal)
        plt.scatter(datax, datay, marker='*', c='#d62728')
    for frame in occ:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.1)

def plotOrientSpeed(startgoal, data):
    fig = plt.figure()
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    plt.scatter(data[:, 2], data[:,3])
    plt.scatter(startgoal[2], startgoal[3])
    plt.scatter(startgoal[6], startgoal[7])
    
def plotAlpha(alpha):
    fig = plt.figure()
    alpha = alpha[0,:]
    alpha=alpha.reshape(4,10,10)
    
    myobj = plt.imshow(alpha[0, :, :])

    for frame in alpha:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.3)

In [18]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
# adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
# occ = adap_pool(occ)
occ = occ.to(device)
data = torch.from_numpy(batch["data"]).to(device)

occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy()
data=data.cpu().detach().numpy()
torch.cuda.empty_cache()

plotData(occ, startgoal, data)

445954


In [19]:
occ.shape

(1, 1, 25, 300, 1853)